In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

import src
from src.bert import training
from src.bert.dataset import PBertDataset
from src.bert.dataset.strategies import MLMin1PopIdeol

In [ ]:
EXCLUDE_CODERS: list[str] = []
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL = "bert-base-multilingual-cased"
BATCH_SIZE = 8
N_EPOCHS = 6
LR = 1e-5
WEIGHT_DECAY = 0.010

THRESHOLDS = {0: 0.112633, 1: 0.575459, 2: 0.388344, 3: 0.152591}

In [ ]:
train = PBertDataset.from_disk(
    path=src.PATH / "data/bert/train.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

test = PBertDataset.from_disk(
    path=src.PATH / "data/bert/test.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
collate_fn = train.create_collate_fn(tokenizer)

train_loader = DataLoader(train, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test, collate_fn=collate_fn, batch_size=64, shuffle=False)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=train.num_labels).to(
    DEVICE
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    amsgrad=False,
    weight_decay=WEIGHT_DECAY,
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20,
    eta_min=LR / 10,
)

for epoch in range(1, N_EPOCHS + 1):
    train_loss = training.train_epoch(model, train_loader, optimizer, lr_scheduler, clip=5)
    eval_loss, score, _ = training.eval_epoch(model, test_loader)
    print(f"{epoch=} {train_loss=:.4f} {eval_loss=:.4f} {score=:.4f}")

epoch=1 train_loss=360.3420 eval_loss=9.7026 score=0.3780


epoch=2 train_loss=281.2398 eval_loss=8.9324 score=0.5572


epoch=3 train_loss=233.7071 eval_loss=9.2197 score=0.6073


epoch=4 train_loss=189.8062 eval_loss=9.3144 score=0.5962


epoch=5 train_loss=155.6215 eval_loss=10.4810 score=0.5709


epoch=6 train_loss=124.8986 eval_loss=11.0570 score=0.5777


In [ ]:
def apply_thresh(y_proba, thresholds: dict):
    y_proba = y_proba.copy()
    for dim, thresh in thresholds.items():
        y_proba[:, dim] = np.where(y_proba[:, dim] > thresh, 1, 0)
    return y_proba

In [ ]:
with torch.inference_mode():
    y_true = []
    y_pred = []
    for batch in test_loader:
        encodings = batch["encodings"]
        encodings = encodings.to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        out = model(**encodings)
        preds = torch.nn.functional.sigmoid(out.logits)
        y_true.extend(batch["labels"].numpy())
        y_pred.extend(preds.cpu().numpy())
    y_pred_05 = np.where(np.array(y_pred) > 0.5, 1, 0)
    y_pred_thresh = apply_thresh(np.array(y_pred), THRESHOLDS)
    y_true = np.array(y_true)

In [ ]:
print(classification_report(y_true, y_pred_05, target_names=["elite", "pplcentr", "left", "right"]))

              precision    recall  f1-score   support

       elite       0.76      0.66      0.71       648
    pplcentr       0.63      0.55      0.59       322
        left       0.67      0.44      0.53       279
       right       0.79      0.35      0.49       155

   micro avg       0.71      0.56      0.62      1404
   macro avg       0.71      0.50      0.58      1404
weighted avg       0.72      0.56      0.62      1404
 samples avg       0.32      0.29      0.29      1404



/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(
    classification_report(
        y_true, y_pred_thresh, target_names=["elite", "pplcentr", "left", "right"]
    )
)

              precision    recall  f1-score   support

       elite       0.69      0.79      0.74       648
    pplcentr       0.65      0.52      0.58       322
        left       0.64      0.48      0.55       279
       right       0.61      0.48      0.54       155

   micro avg       0.67      0.63      0.65      1404
   macro avg       0.65      0.57      0.60      1404
weighted avg       0.66      0.63      0.64      1404
 samples avg       0.35      0.33      0.33      1404



/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/bwfor/home/st/st_st/st_ac138201/bert_populism/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save_pretrained(src.PATH / "tmp/mbert_model")